# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [4]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [5]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [6]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [7]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [8]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [9]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [10]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [11]:
## Data Definition Language 
query = "CREATE TABLE IF NOT EXISTS artist_song_length"
query = query + "(sessionId int, itemInSession int, artist varchar, song varchar, length float , PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
## Data Load
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_length (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line['sessionId']), int(line['itemInSession']), line['artist'], line['song'], float(line['length'])))
print("Finished")

Finished


#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
## Sample Data 
query = "SELECT artist, song, length FROM artist_song_length WHERE sessionId = 338 AND itemInSession  = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [14]:
## Data Definition Language 
query = "CREATE TABLE IF NOT EXISTS artist_song_user"
query = query + "(userid int, sessionId int, itemInSession int, artist varchar, song varchar, firstName varchar, lastName varchar, PRIMARY KEY ((userid, sessionId), itemInSession))"
query = query + "WITH CLUSTERING ORDER BY (itemInSession ASC);"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
## Data Load

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_user (userid, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line['userId']), int(line['sessionId']), int(line['itemInSession']), line['artist'], line['song'], line['firstName'], line['lastName']))
print("Finished")

Finished


In [16]:
## Sample Data 
query = "SELECT artist, song, firstname, lastname FROM artist_song_user WHERE  userid = 10 and sessionid = 182"
query = query + "ORDER BY itemInSession DESC"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ( row.artist, row.song, row.firstname, row.lastname)

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Down To The Bone Keep On Keepin' On Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

The User_id and the song were used as the primary key 

In [17]:
query = "CREATE TABLE IF NOT EXISTS user_song"
query = query + "(user_id int, song varchar, firstName varchar, lastName varchar, PRIMARY KEY (song))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [18]:
## Data Load 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (user_id, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line['userId']), line['song'], line['firstName'], line['lastName']))
print("Finished")

Finished


In [19]:
query = "SELECT song, lastname, firstname FROM user_song WHERE Song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song,  row.lastname, row.firstname,)

All Hands Against His Own Lynch Jacqueline
All Hands Against His Own Levine Tegan
All Hands Against His Own Johnson Sara


### Drop the tables before closing out the sessions

In [1]:
try:
    session.execute( "DROP TABLE IF EXISTS artist_song_length")
    session.execute( "DROP TABLE IF EXISTS artist_song_user")
    session.execute( "DROP TABLE IF EXISTS user_song")
except Exception as e:
    print(e)

name 'session' is not defined


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()